In [113]:
import forallpeople
forallpeople.environment('structural', top_level=True)
from math import sqrt
import streamlit as st
from handcalcs.decorator import handcalc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import handcalcs.render

In [114]:
filename = "W_CISC.xlsm"

In [115]:
df = pd.read_excel(filename,skipfooter =2)

In [116]:
df

,Prp,Ds_m,Avl,Shp,Grp,Use,D,B,T,W,...,J,Cw,Wn,Sw,Qf,Qw,SA,Ds_i,Dn_i,Wt_i
0,*,W1100x607,*,1,NaN,3,1138,410,55.0,31.0,...,57900.0,186000.00,111000,626.000,5640.0,16300.0,3.850,W44x408,44,408.0
1,NaN,W1100x548,*,1,NaN,3,1128,407,50.0,28.0,...,43400.0,164000.00,110000,558.000,5110.0,14700.0,3.830,W44x368,44,368.0
2,NaN,W1100x499,*,1,NaN,3,1118,405,45.0,26.0,...,31100.0,144000.00,109000,495.000,4580.0,13200.0,3.800,W44x335,44,335.0
3,NaN,W1100x433,*,1,NaN,3,1108,402,40.0,22.0,...,21200.0,124000.00,107000,431.000,4060.0,11500.0,3.780,W44x290,44,290.0
4,NaN,W1100x390,*,1,NaN,3,1100,400,36.0,20.0,...,15600.0,109000.00,106000,383.000,3640.0,10300.0,3.760,W44x262,44,262.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,NaN,W150x14,a,1,NaN,2,150,100,5.5,4.3,...,16.8,4.79,3610,0.497,19.0,50.1,0.691,W6x9,6,9.0
285,NaN,W150x13,*,1,NaN,2,148,100,4.9,4.3,...,13.6,4.19,3580,0.438,16.8,45.3,0.687,W6x8.5,6,8.5
286,*,W130x28,*,1,NaN,1,131,128,10.9,6.9,...,127.0,13.80,3840,1.340,39.6,94.1,0.760,W5x19,5,19.0
287,NaN,W130x24,*,1,NaN,1,127,127,9.1,6.1,...,76.2,10.80,3740,1.080,32.4,77.2,0.750,W5x16,5,16.0


In [117]:
def sort_based_on_availablity(sorted_df):
    '''
    Sort database based on availability
    '''
    mask = sorted_df.loc[:,"Avl"]== 'a'                                                  # This function will only store all the W sections that are readily available in Canada and commonly used
    df_sort_availability = sorted_df.loc[mask,["Ds_i","Ds_m","D","B","T","W","BT","HW",
                        "A_Th","Ix","Sx","Zx","Iy",
                       "Sy","Zy","J","Cw","Mass"]]
    return df_sort_availability

In [118]:
def sort_based_on_mass(df_sort_availability,Required_inertia = 1):
    '''
    Sort database based on required moment of inertia and mass
    '''
    mask = df_sort_availability.loc[:,"Ix"]>=Required_inertia                                                # This function will only store all the W sections that are readily available in Canada and commonly used
    df_sort = df_sort_availability.loc[mask,["Ds_i","Ds_m","D","B","T","W","BT","HW",
                        "A_Th","Ix","Sx","Zx","Iy",
                       "Sy","Zy","J","Cw","Mass"]]
    df_sort_by_mass = df_sort.sort_values("Mass")
    return df_sort_by_mass

In [119]:
df_section_available = sort_based_on_availablity(df)
df_section_available

,Ds_i,Ds_m,D,B,T,W,BT,HW,A_Th,Ix,Sx,Zx,Iy,Sy,Zy,J,Cw,Mass
57,W36x150,W920x223,911,304,23.9,15.9,6.36,54.29,28500,3760.00,8260.0,9520,112.000,738.0,1160.0,4180.0,22100.00,223
58,W36x135,W920x201,903,304,20.1,15.2,7.56,56.76,25600,3250.00,7190.0,8340,94.400,621.0,982.0,2880.0,18400.00,201
70,W33x130,W840x193,840,292,21.7,14.7,6.73,54.19,24700,2780.00,6630.0,7620,90.300,618.0,971.0,3050.0,15100.00,193
71,W33x118,W840x176,835,292,18.8,14.0,7.77,56.96,22400,2460.00,5900.0,6810,78.200,536.0,844.0,2220.0,13000.00,176
85,W30x108,W760x161,758,266,19.3,13.8,6.89,52.13,20400,1860.00,4900.0,5660,60.700,457.0,720.0,2070.0,8280.00,161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,W6x25,W150x37,162,154,11.6,8.1,6.64,17.14,4730,22.20,274.0,310,7.070,91.8,140.0,192.0,40.00,37
280,W6x20,W150x30,157,153,9.3,6.6,8.23,20.97,3790,17.10,218.0,244,5.560,72.6,111.0,100.0,30.30,30
281,W6x15,W150x22,152,152,6.6,5.8,11.52,23.93,2840,12.00,159.0,176,3.870,50.9,77.5,41.5,20.40,22
283,W6x12,W150x18,153,102,7.1,5.8,7.18,23.93,2280,9.15,120.0,136,1.260,24.7,38.2,36.9,6.70,18


In [120]:
Required_inertia = 500

In [121]:
df_sorted_by_mass = sort_based_on_mass(df_section_available,Required_inertia)
df_sorted_by_mass.head(5)

,Ds_i,Ds_m,D,B,T,W,BT,HW,A_Th,Ix,Sx,Zx,Iy,Sy,Zy,J,Cw,Mass
125,W24x55,W610x82,599,178,12.8,10.0,6.95,57.34,10400,560.0,1870.0,2200,12.1,136.0,218.0,488.0,1040.0,82
124,W24x62,W610x92,603,179,15.0,10.9,5.97,52.57,11800,646.0,2140.0,2510,14.4,161.0,258.0,710.0,1250.0,92
141,W21x62,W530x92,533,209,15.6,10.2,6.70,49.20,11800,552.0,2070.0,2360,23.8,228.0,355.0,762.0,1590.0,92
123,W24x68,W610x101,603,228,14.9,10.5,7.65,54.59,13000,764.0,2530.0,2900,29.5,259.0,404.0,781.0,2550.0,101
140,W21x68,W530x101,537,210,17.4,10.9,6.03,46.07,12900,617.0,2300.0,2620,26.9,256.0,400.0,1020.0,1820.0,101


In [122]:
while True:
    try:
        Section = input('Enter section name from sections column:')
    except valueError:
        print('Data Input Error! Try Again')
    else:
        break

Enter section name from sections column: W24x55


In [123]:
mask = df.loc[:,"Ds_i"] == Section

In [124]:
df_section = df.loc[mask,["Ds_i","Ds_m","D","B","T","W","BT","HW",
                        "A_Th","Ix","Sx","Zx","Iy",
                       "Sy","Zy","J","Cw"]]
df_section

,Ds_i,Ds_m,D,B,T,W,BT,HW,A_Th,Ix,Sx,Zx,Iy,Sy,Zy,J,Cw
125,W24x55,W610x82,599,178,12.8,10.0,6.95,57.34,10400,560.0,1870.0,2200,12.1,136.0,218.0,488.0,1040.0
